### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=G-zBkueElVEO

https://ai.google.dev/tutorials/python_quickstart


In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 3.9 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  # Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [3]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [5]:
model = genai.GenerativeModel('gemini-pro')

## Imports

In [6]:
import os
import json
import pandas as pd
import time
import re
import csv
import concurrent.futures

### Data:
Our data will be taken from 162 snort rules that have already been manually labeled to techniques from MITRE ATT&CK.

In [7]:
!git clone https://github.com/trickdeath0/Labeling_IDS_to_MITRE.git

Cloning into 'Labeling_IDS_to_MITRE'...
remote: Enumerating objects: 284, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 284 (delta 151), reused 241 (delta 111), pack-reused 0
Receiving objects: 100% (284/284), 863.29 KiB | 14.15 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [34]:
data = pd.read_csv('/content/Labeling_IDS_to_MITRE/ground_truth.csv')
print(data.head())
rules_list = data['Rule']
true_labels = data['technique ids']

#print(data['Sid'][0+41])
print(f"\n{len(data)=}")

   Unnamed: 0  Sid                                               Rule  \
0           0  213  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
1           1  214  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
2           2  215  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
3           3  216  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
4           4  233  alert tcp $EXTERNAL_NET any -> $HOME_NET 27665...   

  technique ids  
0     ['T1014']  
1     ['T1014']  
2     ['T1014']  
3     ['T1014']  
4     ['T1078']  

len(data)=161


In [15]:
def clean_response(text):
    text = text.data.replace(">", "").strip()  # Remove leading ">", whitespace
    try:
      text = text.replace("```json", "")
      text = text.replace("```", "")
    except:
      pass
    return text


# **Prompting without techniques guide (WTG):**
At this stage, the LLMs will receive a prompt that does not include the list of techniques from MITRE ATT&CK in order to examine the results of the models based on prior knowledge that has been trained. According to our request, the LLMs will classify the techniques according to the content of the rule.


**Prompt1**:

      prompt = f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
      Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
      Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
      Please don't write anything but the JSON. Rule: {snort_rule}"""


**prompt2**:

      prompt2 = f"""I'm going to give you a Snort rule. Read the Snort rule carefully, because I'm going to given you a task about it. Here is the Snort rule: <snort_rule>{snort_rule}</snort_rule>

      First, find the techniques from MITRE ATT&CK that are most relevant to the Snort rule.

      Then, answer the task, for each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.

      Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.

      Thus, the format of your overall response should look like what's shown between the <examples></examples> tags. Make sure to follow the formatting and spacing exactly.


      <examples>
        [
          "sid": "2274",
          "Technique ID": "T1110",
          "Technique name": "Brute Force",
          "Quotes": [
            "\"PROTOCOL-POP login brute force attempt\"",
            "track by_dst,count 30,seconds 30"
          ],
          "Explanation": "The rule is looking for excessive \"USER\" commands within a short period of time, which are common indicators of brute-force attacks targeting the POP3 service."
        ]
        </examples>

        Do not include anything besides write the JSON.
        """


**prompt3**:

        prompt3 = f"""You work in a company that deals with information security, your role in the company is to label techniques from MITRE ATT&CK to the rules of IDS systems. The labeling between a rule and a technique indicates that the attacker operated with a technique that you found to be suitable for the rule that alerted the IDS system. Now we will test your knowledge labeling IDS rules for MITRE ATT&CK techniques. For your task, you're going to have a single Snort IDS rule and you'll need to label the most relevant techniques from MITRE ATT&CK associated with the rule. From the rule you receive, your labeling should be based on your knowledge and the information found within the 'msg' in the rule received. For each technique you call the rule, include the following information as JSON format in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.  Note: The value of the 'Quotes' field should contain quotation marks from the data sets relevant to the mapped technique. The value of the 'Explanation' should be your explanation of why you decided to give the technique and how it relates to the rule. The 'Technique ID' should be the official MITRE technique ID.
        Please don't write anything but the JSON. Rule: {snort_rule}""")


**prompt4**

        prompt4 = f'''You are going to receive a Snort rule and your task is to find as many MITRE ATT&CK techniques as possible that are associated with the rule. Note: You should categorize the techniques to 1 or 2. Technique of type 1 is a technique that you can associate with the rule directly based on the rule. Technique of type 2 is a technique that can be associated with the rule indirectly, based on your knowledge and understanding. The categorization value should be the value 1 or 2, based on the explanation given above. The quotes field value should contain quotes from the rules data that are relevant to the technique mapped and they are the main reason you believe the mapping to this technique is correct. The explanation’s value should be your explanation for why you decided to give the technique and how it is associated with the rule. The technique id should be the official MITRE technique id. For each technique include the following information as JSON: sid, Technique id, Technique name, Categorization, Quotes, Explanation. After each rule I will provide you with, answer according to the provided format. Please do not write anything else but the JSON. Rule: {snort_rule}''')


In [16]:
def wtg(snort_rule):

  prompt = f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
  Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
  Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
  Please don't write anything but the JSON. Rule: {snort_rule}"""

  response = model.generate_content(prompt)

  return response


In [17]:
rule_dict = {}
max_retries = 3  # Maximum number of retries

for index, rule in enumerate(rules_list):
    retries = 0
    while retries < max_retries:
        try:
            res = wtg(rule)
            text = res.text
            # Check if the text contains the desired pattern
            t_numbers = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)
            if t_numbers:  # If the pattern is found
                rule_dict[data['Sid'][index]] = to_markdown(text)
                break  # Break out of the retry loop if successful
            else:
                print("Desired pattern not found in the text. Retrying...")
                retries += 1
                time.sleep(1)  # Wait for a short duration before retrying
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
                time.sleep(1)  # Wait for a short duration before retrying
            else:
                print("Max retries reached. Unable to process this rule.")

# If sending fails, attempt to send again
try:
    # Code to send data
    pass
except Exception as e:
    print(f"Sending failed: {e}")
    # Retry sending here


Write to csv

In [18]:

# Define the field names
field_names = ["Sid", "Response", "Technique_id", "True_labels"]

# Open the CSV file in write mode (truncating any existing content)
with open("prompting_without_techniques_guide.csv", "w", newline="") as csvfile:
    # Create a DictWriter object with the specified field names
    writer = csv.DictWriter(csvfile, fieldnames=field_names)

    # Write the header row
    writer.writeheader()

    # Extract relevant data from each item and write it as a dictionary
    counter = 0
    for key, value in rule_dict.items():
      text = clean_response(value)
      technique_ids = []

      if "'Sid" in text:
        # Define a regex pattern to switch single quotes to double quotes
        pattern = re.compile(r"((^|\s)'((?:[^'\\]|\\.)*)'(?=[\s.,:;!?)]))|(:\s*'((?:[^'\\]|\\.)+)')")
        # Switch single quotes to double quotes
        text = pattern.sub(lambda x: x.group().replace("'", '"'), text)

        pattern = re.compile(r'"\S+"[\s\.]|\s"[\w\s]*"\s')
        text = re.sub(pattern, "", text)

      # Extracting "TXXXX" numbers using regular expression
      technique_ids = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)

      # Extracting "Sid"
      match = re.search(r'[\'\"][s|S]id[\'\"]: [\'\"](\d+)[\'\"]', text)
      if match:
          sid_number = match.group(1)


      # Assuming each item has all necessary fields:
      insertRow = {
          "Sid": sid_number,
          "Response": text,
          "Technique_id": technique_ids,  # Handle potential absence
          "True_labels": true_labels[counter],
      }
      writer.writerow(insertRow)
      counter += 1


# Evaluation


*   Persicion
*   Recall
*   F-1



In [19]:
import ast
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def evaluation(true_labels, predicted_labels):

  recall = []
  precision = []
  f1 = []


  for i in range(len(true_labels)):
    trueList = ast.literal_eval(true_labels[i])
    predList = ast.literal_eval(predicted_labels[i])
    # Extract only the 'TXXXX' part from each string in the list
    predList = [item.split('.')[0] if '.' in item else item for item in predList]
    intersection = set(trueList).intersection(set(predList))
    #print(list(intersection))

    if (len(predList) != 0):
      recall.append(len(intersection) / len(trueList))
      precision.append(len(intersection) / len(predList))
      try:
        f1.append((2 * precision[i] * recall[i]) / (recall[i] + precision[i]))
      except:
        f1.append(0)

  # Avg.
  average_recall = sum(recall) / len(recall)
  average_precision = sum(precision) / len(precision)
  average_f1 = sum(f1) / len(f1)

  print("Metric    |   Score")
  print("-------------------")
  print(f"Precision |   {average_precision:.2f}")
  print(f"Recall    |   {average_recall:.2f}")
  print(f"F1 Score  |   {average_f1:.2f}")

In [20]:
loadData = pd.read_csv('prompting_without_techniques_guide.csv')
true_labels2 = loadData['True_labels']
predicted_labels = loadData['Technique_id']

evaluation(true_labels2, predicted_labels)


Metric    |   Score
-------------------
Precision |   0.14
Recall    |   0.12
F1 Score  |   0.11


# **Prompting with techniques guide (TG):**
In the next step, we will provide the LLMs with the list of all the techniques from MITRE ATT&CK, to guarantee that the models are targeted to the present techniques, even the infrequently used ones. Each technique will include the technique number, the name of the technique and its description. The techniques will be provided to the models in the form of batches (due to the memory limit of the models) and after each batch we will ask him to classify the appropriate techniques from the list he received (if exist), finally we will unite the model's answers for each individual rule.


pre colletion data for TG

In [21]:
def recursive_enter(path: str, file_list: list = None) -> list:
    if file_list is None:
        file_list = []

    try:
        os.chdir(path)  # Change path

        items = os.listdir()  # List everything in the directory
        for item in items:
            full_path = os.path.join(path, item)

            if full_path.endswith(".json"):
                with open(full_path) as f:
                    file_list.append(json.load(f))

    except Exception as e:
        print(f"An error occurred: {e}")

    return file_list

tacticFolder = "/content/Labeling_IDS_to_MITRE/Extract data from MITRE ATTACK/techniques_split"
file_list = []
MITRE_Technique = recursive_enter(tacticFolder, file_list)
print(len(MITRE_Technique))
os.chdir("/content/")

11


In [53]:
def tg(snort_rule, techniques):

  prePrompt = f"""You are an information security expert. Now I will provide you information about techniques from MITRE ATT&CK, you will use the information for a task you will receive later. Do not reply to the information you receive."""

  dataPrompt = f"The information:\n {str(techniques)}"

  response_data = f"""Your task is to label IDS rules for MITRE ATT&CK techniques based on the information I have provided you. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques associated with the rule from the information I provided you only.
   Note 1: There is not necessarily a suitable technique in the information, return a technique if and only if it has an unambiguous relationship to the provided rule, if not return an empty JSON. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
   Note 2: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
   Please don't write anything but the JSON. Rule: {snort_rule}"""

  tg_data_list = [prePrompt, dataPrompt, response_data]

  try:
    response = model.generate_content(tg_data_list)
    #print(response.parts)
  except:
    #print(response.parts)
    response = model.generate_content("Return empty JSON {}")
  finally:
    return response




In [29]:
import csv
import re

headersCSV_TG = ["Sid", "Response", "Technique_id", "True_labels"]

# Initial write to csv with header
with open('prompting_with_techniques_guide.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)
    writer.writeheader()

def appendToCSV(rows_data, counter) -> None:
    '''
    rows_data -> {213: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, ...]}
    '''
    # Open the CSV file in append mode to add new rows
    with open('prompting_with_techniques_guide.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)

        # Loop through each row and write data
        for row, value in rows_data.items():
            technique_ids = []
            response_text = ""
            for i in value:
                text = clean_response(i)
                response_text += text
                #print(text)

                try:
                    # Extracting "TXXXX" numbers using regular expression
                    technique_ids.extend(re.findall(r': [\'\"](T\d+(?:\.\d+)?)', text))
                except Exception as e:
                    print(f"Error extracting technique IDs: {e}")

            insertRow = {
                "Sid": row,
                "Response": response_text,
                "Technique_id": technique_ids,
                "True_labels": true_labels[counter]
            }

            # Write the row to the CSV file
            writer.writerow(insertRow)


In [56]:
def tg_split_data(rules_list_index, index):

  for rule in rules_list_index:
    print(f"index {index} \t Sid: {data['Sid'][index]}")

    tg_dict = {}
    count = 0 #####
    for batch in MITRE_Technique: # 11 files
      res = tg(rule, batch)
      sid = data['Sid'][index]
      if sid not in tg_dict:
        tg_dict[sid] = []
        tg_dict[sid].append(to_markdown(res.text))
      else:
        #print(res.text)
        try:
          tg_dict[sid].append(to_markdown(res.text))
        except:
          tg_dict[sid].append(to_markdown("{}"))
      print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
      #print(to_markdown(res.text))
      count += 1 #######

    # Write to csv
    appendToCSV(tg_dict, index)
    index += 1

In [57]:
#rules_list1 = rules_list[:20] # index 0-19
#rules_list2 = rules_list[20:80] # index 20-79
# rules_list3 = rules_list[80:] # index 80-162

#tg_split_data(rules_list1, 0)
#tg_split_data(rules_list2, 20)
# tg_split_data(rules_list3, 80)

rules_list3 = rules_list[100:]
tg_split_data(rules_list3, 100)

index 100 	 Sid: 30569
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~3~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~4~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~5~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~6~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~7~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~8~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~9~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
index 101 	 Sid: 31070
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~

# Evaluation


*   Persicion
*   Recall
*   F-1



In [58]:
loadData = pd.read_csv('prompting_with_techniques_guide.csv')
true_labels2 = loadData['True_labels']
predicted_labels = loadData['Technique_id']

evaluation(true_labels2, predicted_labels)


Metric    |   Score
-------------------
Precision |   0.09
Recall    |   0.24
F1 Score  |   0.01


# Original

## Same code without split data (long running)

In [ ]:
for index, rule in enumerate(rules_list[:2]): # 162 rules
  print(f"index {index} \t Sid: {data['Sid'][index]}")

  # Splice from index 1 to index 3 (exclusive)
  start_index = 0
  end_index = 3
  spliced_dict = {k: MITRE_Technique[k] for k in list(MITRE_Technique.keys())[start_index:end_index]}


  tg_dict = {}
  count = 0 #####
  for tactics, techniques_inside in spliced_dict.items(): # 14 folders  #################### return to MITRE_Technique.items()
    print(f"Tactic check: {tactics}")
    for techniques in techniques_inside:
      print(f"inside folder tactic: {techniques}\n")
      res = tg(rule, tactics, techniques)
      sid = data['Sid'][index]
      if sid not in tg_dict:
        tg_dict[sid] = []
        tg_dict[sid].append(to_markdown(res.text))
      else:
        tg_dict[sid].append(to_markdown(res.text))
      print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
      print(to_markdown(res.text))
      count += 1 #######

  # Write to csv
  appendToCSV(tg_dict, index)


In [ ]:
#print(to_markdown(res.text).data)

print(tg_dict)
# print(tg_dict[213][0].data)
for key, value in tg_dict.items():
  for i in value:
    text = clean_response(i)
    print(text)

{213: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>]}
{}
{}
{}
{}


In [ ]:
to_markdown(res.text)

> ```json
> [
>   {
>     "sid": "214",
>     "Technique ID": "T1036",
>     "Technique Name": "Masquerading",
>     "Quotes": [
>       "Linux rootkit attempt lrkr0x"
>     ],
>     "Explanation": "The rule triggers on a connection to port 23 (Telnet) from an external network to Telnet servers, and it looks for the string 'lrkr0x' in the content of the connection. This is often associated with attempts to install a rootkit on Linux systems, which is a form of Masquerading, where malicious software attempts to disguise itself as legitimate software or processes."
>   }
> ]
> ```